# Example: FEniCS with Docker
[Adapted from BET Documentation](http://ut-chg.github.io/BET/examples/example_rst_files/FEniCS.html#fenicsexample)

We will walk through the following example. This example will only run in serial using serial runs of a model (described below). If the user takes Steps (0)-(3) and runs them in a separate script, then the saved discretization object can be loaded into a different script containing Steps (4)-(5) (and optionally including Step (6)) to solve the stochastic inverse problem in parallel using BET. To see an example describing how to run multiple instances of the (serial) model with different parameters, see Example: Multiple Serial FEniCS for more information.


This example requires the following external packages not shipped with BET:

* An installation of Docker
* A pulled image of FEniCS via Docker

For more information on how to install docker on you computer, visit [the Docker website](https://www.docker.com/). Instructions on how to pull a working version of FEniCS via Docker can be [found here](http://fenics.readthedocs.io/projects/containers/en/latest/index.html).

This example generates samples for a KL expansion associated with a covariance defined by `cov` in computeSaveKL.py on an L-shaped mesh that defines the permeability field for a Poisson equation solved in myModel.py.

The quantities of interest (QoI) are defined as two spatial averages of the solution to the PDE.

The user defines the dimension of the parameter space (corresponding to the number of KL terms) and the number of samples in this space.

Docker allows BET to utilize the FEniCS PDE solver without having to deal with several of the more arduous customizations required to set up FEniCS on the user's personal machine. In myModel_Interface.py, a Docker container is created and then all the FEniCS calculations are executed in the container, afterwhich the container is closed. Because of the portability and customizability of Docker containers, this example provides a framework for how BET could potentially be linked to all sorts of other computational models which require specific computational software.

![Placeholder for actual picture visualizing BET / Docker / FEniCS container interaction](temp_pic.png)

Even though we are coupling to the state-of-the-art FEniCS code for solving a PDE, we again see that the actual process for solving the stochastic inverse problem is quite simple requiring a total of 5 steps with BET excluding any post-processing the user may want. In general the user will probably not write code with various options as was done here for pedagogical purposes. We break down the actual example included with BET step-by-step below, but first, to showcase the overall simplicitly, we show the “entire” code (omitting setting the environment, post-processing, and commenting) required for solving the stochastic inverse problem using some default options:

```python
sampler = bsam.sampler(my_model)

num_KL_terms = 2
computeSaveKL(num_KL_terms)
input_samples = samp.sample_set(num_KL_terms)
KL_term_min = -3.0
KL_term_max = 3.0
input_samples.set_domain(np.repeat([[KL_term_min, KL_term_max]],
                                   num_KL_terms,
                                   axis=0))
input_samples = sampler.regular_sample_set(input_samples, num_samples_per_dim=[10, 10])
input_samples.estimate_volume_mc()

my_discretization = sampler.compute_QoI_and_create_discretization(input_samples)

param_ref = np.ones((1,num_KL_terms))
Q_ref = my_model(param_ref)
simpleFunP.regular_partition_uniform_distribution_rectangle_scaled(
    data_set=my_discretization, Q_ref=Q_ref[0,:], rect_scale=0.1,
    cells_per_dimension=3)

calculateP.prob(my_discretization)
```

## Step (0): Setting up the environment
Import the necessary modules:

In [1]:
import numpy as np
import bet.calculateP.simpleFunP as simpleFunP
import bet.calculateP.calculateP as calculateP
import bet.postProcess.plotP as plotP
import bet.postProcess.plotDomains as plotD
import bet.sample as samp
import bet.sampling.basicSampling as bsam
import subprocess as sp
import os

## Step (1): Define interface to the model
Import the Python script interface to the model using FEniCS that takes as input a numpy array of model input parameter samples, generated from the sampler (see below), creates or starts appropriate Docker container, and executes the FEniCS scripts to evaluate the model and generate QoI samples:

In [2]:
from myModel_Interface import bet_docker_interface

Define the sampler that will be used to create the discretization object, which is the fundamental object used by BET to compute solutions to the stochastic inverse problem. The sampler and my_model is the interface of BET to the model, and it allows BET to create input/output samples of the model:

In [3]:
sampler = bsam.sampler(bet_docker_interface)

## Step (2): Describe and sample the input space
We compute and save the KL expansion once so that this part, which can be computationally expensive, can be done just once and then commented out for future runs of the code using the same set of KL coefficients defining the parameter space.

In [4]:
# Choose the number of KL terms
num_KL_terms = 2

### Creating a Docker Container and Running KL Expansion

Since the KL expansion requires FEniCS, we can create a FEniCS docker container to execute the KL expansion script. After we create the container, we can run the script in the container just once.

The following checks to make sure docker is installed and running on the user's computer by checking the Docker version. If it is not working, then it will raise an error.

In [5]:
# checks to make sure docker is installed and running correctly
try:
    dockercommand = ("docker version")
    print(sp.check_output(dockercommand.split(" ")))
except:
    print("\n Error Running Docker: \n Check that Docker "+
          "is properly installed and currently running \n")
    raise

Client:
 Version:      17.06.2-ce
 API version:  1.30
 Go version:   go1.8.3
 Git commit:   cec0b72
 Built:        Tue Sep  5 19:57:19 2017
 OS/Arch:      windows/amd64

Server:
 Version:      17.06.2-ce
 API version:  1.30 (minimum version 1.12)
 Go version:   go1.8.3
 Git commit:   cec0b72
 Built:        Tue Sep  5 19:59:19 2017
 OS/Arch:      linux/amd64
 Experimental: true



The following code creates the FEniCS Docker container. It shares the local directory as a "volume" with the container so that scripts in the current directory can be run inside the container and the results saved on the local machine.  For details about the options and arguments, see the [Docker documentation](https://docs.docker.com/engine/reference/commandline/run/).

> **Note**: on Windows, make sure appropriate drives have been shared via Docker for Windows settings.

In [6]:
# get working directory and define container name
localdirect = os.getcwd()
containername = ("ComputeKL_FEniCS")

In [7]:
# docker create command string
dockercreate = ("docker create -i --name "+containername
                        + " -w /home/fenics/shared" # sets working directory
                 +" -v "+localdirect+":/home/fenics/shared" # share current dir.
                  +" quay.io/fenicsproject/stable") # name of parent image 

#print(dockercreate)

# use subprocess to run command string and check output
out = sp.check_output(dockercreate.split(" "))

Next we start the container and execute the script [Compute_Save_KL.py](Compute_Save_KL.py). Note we also add the optional argument "num_KL_terms" to the script execution. The last few lines stops the container. 

*Note: Removing the container is optional, but note that you cannot create two containers with the same name.*

In [8]:
# name of the python script which runs the fenics model
fenics_script = ("Compute_Save_KL.py "+str(num_KL_terms))
    
# starts container
dockerstart = ("docker start "+containername)
outstatus = sp.check_output(dockerstart.split(" "))
print(outstatus+" container has started...")

ComputeKL_FEniCS
 container has started...


In [9]:
# execute python script in FEniCS container
dockerexec = ("docker exec "+containername+" python "+fenics_script)

outstatus = sp.Popen(dockerexec.split(" "),stdout=sp.PIPE)
print(outstatus.communicate()[0])
    

2
which is type: <type 'int'>
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling DOLFIN just-in-time (JIT) compiler, this may take some time.
---------------------------
---------------------------
 Building Covariance Matrix
---------------------------
---------------------------
---------------------------
---------------------------
 Finished Covariance Matrix
---------------------------
---------------------------
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
---------------------------
---------------------------
 Building Mass Matrix 
---------------------------
---------------------------
---------------------------
---------------------------
 Finished Mass Matrix 
---------------------------
---------------------------
Calling FFC just-in-time (JIT) compiler, this may take some time.
  Ignoring precision in integral metadata compiled using quadrature representation. Not

In [10]:
# close docker container
dockerclose = ("docker stop "+containername)
outstatus = sp.check_output(dockerclose.split(" "))
print(outstatus+" container has closed.")


ComputeKL_FEniCS
 container has closed.


In [11]:
# remove docker container if needed
#dockerRemove = ("docker rm "+containername)
#outstatus = sp.check_output(dockerRemove.split(" "))
#print(outstatus+" has been removed.")


The KL Expansion should now be computed.

### Initializing the Sampler
Now we can initialize the parameter space and assume that any KL coefficient belongs to the interval [-3.0,3.0]:

In [12]:
input_samples = samp.sample_set(num_KL_terms)
KL_term_min = -3.0
KL_term_max = 3.0
input_samples.set_domain(np.repeat([[KL_term_min, KL_term_max]],
                               num_KL_terms,
                               axis=0))

### Suggested changes for user exploration (1):
Try with and without random sampling.

If using **regular** sampling, try different numbers of samples per dimension (note that if `num_KL_terms` is not equal to 2, then the user needs to be careful using regular sampling):

In [13]:
randomSampling = False
if randomSampling is True:
    input_samples = sampler.random_sample_set('random', input_samples, 
                                              num_samples=1E2)
else:
    input_samples = sampler.regular_sample_set(input_samples, 
                                            num_samples_per_dim=[10, 10])

### Suggested changes for user exploration (2):
A standard Monte Carlo (MC) assumption is that every Voronoi cell has the same volume. If a regular grid of samples was used, then the standard MC assumption is true.

See what happens if the MC assumption is not assumed to be true, and if different numbers of points are used to estimate the volumes of the Voronoi cells:

In [14]:
MC_assumption = True
if MC_assumption is False:
    input_samples.estimate_volume(n_mc_points=1E5)
else:
    input_samples.estimate_volume_mc()

## Step (3): Generate QoI samples
Create the discretization object holding all the input (parameter) samples and output (QoI) samples using the sampler:

In [15]:
my_discretization = sampler.compute_QoI_and_create_discretization(
                            input_samples, savefile='FEniCS_Example.txt.gz')

Container named 'BET_to_FEniCS' not found. Creating new Docker container...
New container created named: BET_to_FEniCS
BET_to_FEniCS
 container has started...
Calling FFC just-in-time (JIT) compiler, this may take some time.

BET_to_FEniCS
 container has closed.


IOError: [Errno 2] No such file or directory: 'QoI_outsample_values.npy'

At this point, all of the model information has been extracted for BET (with the possibly exception of evaluating the model to generate a reference QoI datum or a distribution of the QoI), so the model is no longer required for evaluation. The user could do Steps (0)-(3) in a separate script, and then simply load the discretization object as part of a separate BET script that does the remaining steps. When the model is expensive to evaluate, this is an attractive option since we can now solve the stochastic inverse problem (with many different distributions defined on the data space) without ever having to re-solve the model (so long as we are happy with the resolution provided by the current discretization of the parameter and data spaces).